In [220]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib as plt

%matplotlib inline

In [223]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10466 entries, 0 to 10465
Data columns (total 3 columns):
PC_CID      10466 non-null float64
Activity    10466 non-null float64
Smiles      10466 non-null object
dtypes: float64(2), object(1)
memory usage: 245.4+ KB


In [222]:
full_df = pd.read_csv('/Users/marenaisabelle/CODE/GADSI/fulldf')

In [205]:
x = [x.strip() for x in full_df.Smiles.values]
y = full_df.Activity.values

In [215]:
active_df = full_df[full_df['Activity'] == 1]
print active_df.shape

(241, 4)


In [207]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem

mols, np_fps = [], []

# generate fmolecules
for m in x:
    mol = Chem.MolFromSmiles(m)
    mols.append(mol)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    np_fps.append(arr)

In [211]:
len(np_fps)

10466

In [239]:
rf = RandomForestClassifier(n_estimators=100, criterion='gini', max_features=100)
scores = cross_val_score(rf, np_fps, y, cv=10)
print np.mean(scores)
print rf.estimators_

0.97391627156
[]


In [209]:
rf = RandomForestClassifier(n_estimators=10, criterion='gini', max_features=100)
X_train, X_test, y_train, y_test = train_test_split(np_fps, y, test_size = 0.4)
rf.fit_transform(X_train, y_train)

Y_pred = rf.predict(X_test)
print confusion_matrix(y_test, Y_pred)

/Users/marenaisabelle/anaconda/envs/my-rdkit-env/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


[[4094    4]
 [  77   12]]


In [237]:
from sklearn.ensemble import IsolationForest

isof = IsolationForest()
isof.fit(np_fps)
ifdf = isof.decision_function(np_fps)
print pd.Series(ifdf).describe()

two_std = np.mean(ifdf) - 2*np.std(ifdf)
anomalies = [x for x in ifdf if x<two_std]
print len(anomalies)

count    10466.000000
mean         0.175199
std          0.009646
min          0.085743
25%          0.169673
50%          0.176039
75%          0.181855
max          0.198278
dtype: float64
327


In [236]:
X_train, X_test, y_train, y_test = train_test_split(np_fps, y, test_size = 0.4)
ytrain_act = y_train[y_train ==1]
print "Size of train set : %d" % (len(y_train))
print "Active Compounds in train: %d" % (len(ytrain_act))

isof = IsolationForest()
isof.fit(X_train, y_train)
tst = isof.decision_function(X_test)
pd.Series(tst).describe()
print "Number of Anomalies: %d" % (len(anomalies))

Size of train set : 6279
Active Compounds in train: 143
Number of Anomalies: 298
